# New Covid-19 Cases by State
### Date: 8/5/19
In this notebook I'll be depicting the new covid-19 cases by state in a dynamic bar chart. Dataset is provided from NY Times and sourced from  https://www.kaggle.com/fireballbyedimyrnmom/us-counties-covid-19-dataset

In [2]:
import pandas as pd
import datetime

In [3]:
# import csv file
data = pd.read_csv('./dataset/us-counties.csv')

In [4]:
# sneak peak at data
data.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0
1,2020-01-22,Snohomish,Washington,53061.0,1,0
2,2020-01-23,Snohomish,Washington,53061.0,1,0
3,2020-01-24,Cook,Illinois,17031.0,1,0
4,2020-01-24,Snohomish,Washington,53061.0,1,0


In [18]:
# size of dataset
print(f'Size of dataset: {data.shape[0]} rows\n')

# see which dates are available in the dataset
min_date = data.date.min()
max_date = data.date.max()
print(f'Min date: {min_date}')
print(f'Max date: {max_date}\n')



Size of dataset: 402068 rows

Min date: 2020-01-21
Max date: 2020-08-04



In [102]:
# group data set by state and date
states = data.groupby(['state','date']).sum()[['cases']]
states.columns = ['cum_cases']

In [103]:
states.head()

cum_cases
state   date                 
Alabama 2020-03-13          6
        2020-03-14         12
        2020-03-15         23
        2020-03-16         29
        2020-03-17         39

In [188]:
# create new dataframe with all dates in dataset
dates = []

start_date = datetime.datetime.strptime(min_date, '%Y-%m-%d')
end_date = datetime.datetime.strptime(max_date, '%Y-%m-%d')

num_days = int((end_date - start_date).days) + 1

for _ in range(num_days):
    if len(dates) == 0:
        dates.append(start_date)
    else:
        date = dates[-1] + datetime.timedelta(days = 1)
        dates.append(date)

df = pd.DataFrame(index=dates)

# for each state, add column with cumulative cases
state_names = data.state.unique()

states_df = states.reset_index()
states_df['date'] = pd.to_datetime(states_df['date'], format='%Y-%m-%d')
states_df = states_df.set_index('date')

for date, row in states_df.iterrows():
    if row['state'] not in df.columns:
        df[row['state']] = 0

    df.at[date, row['state']] = row.cum_cases

# transpose dataframe 
df_transpose = df.T
df_transpose.head()

# rename columns as Month - Day - Year
dates_str = [d.strftime('%b-%d-%Y') for d in dates]
df_transpose.columns = dates_str

In [190]:
# export to excel load to load to Flourish for racing bar chart
df_transpose.to_excel('./dataset/output_data.xls')